In [385]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

import requests

```
accuracy = eval_clf_performance(args)
    cosine_distance = eval_mapper_performance(args)
    accuracy_score = (
        0
        if accuracy < BASE_ACCURACY - ACCURACY_DROP_THRESHOLD
        else (
            1
            if accuracy > BASE_ACCURACY
            else (accuracy - BASE_ACCURACY + ACCURACY_DROP_THRESHOLD)
            / ACCURACY_DROP_THRESHOLD
        )
    )
return accuracy_score + cosine_distance
```

In [216]:
data = np.load(
    "../data/DefenseTransformationEvaluate.npz"
)
print(data["labels"], data["representations"].shape)

data_sub = np.load("../data/DefenseTransformationSubmit.npz")
print(data_sub["representations"].shape)

[7 8 5 ... 0 5 7] (5000, 192)
(20250, 192)


In [335]:
# Load data from the provided file
data = np.load('../data/DefenseTransformationEvaluate.npz')
X = data['representations']
y = data['labels']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier on clean representations
classifier_clean = MLPClassifier()
classifier_clean.fit(X_train, y_train)

# Evaluate classifier performance on clean representations
y_pred_clean = classifier_clean.predict(X_test)
accuracy_clean = accuracy_score(y_test, y_pred_clean, normalize = True)
print("Accuracy on clean representations:", accuracy_clean)

Accuracy on clean representations: 0.935


In [221]:
def apply_defense_transformation(X):
    # random transformation
    np.random.seed(0)
    X = X + np.random.normal(0, 1, X.shape)
    return X

In [222]:
def evaluate_model(transformation_fuction, dataset, model):
    # Apply defense transformation (e.g., your transformation)
    transformed_X_train = apply_defense_transformation(X_train)
    transformed_X_test = apply_defense_transformation(X_test)

    # Train a classifier on transformed representations
    classifier_transformed = MLPClassifier()
    classifier_transformed.fit(transformed_X_train, y_train)

    # Evaluate classifier performance on transformed representations
    y_pred_transformed = classifier_transformed.predict(transformed_X_test)
    accuracy_transformed = accuracy_score(y_test, y_pred_transformed)
    print("Accuracy on transformed representations:", accuracy_transformed)

Accuracy on transformed representations: 0.931


In [262]:
# Apply defense transformation (e.g., your transformation)
transformed_X_train = apply_defense_transformation_2(X_train)
transformed_X_test = apply_defense_transformation_2(X_test)

# Train a classifier on transformed representations
classifier_transformed = MLPClassifier()
classifier_transformed.fit(transformed_X_train, y_train)

# Evaluate classifier performance on transformed representations
y_pred_transformed = classifier_transformed.predict(transformed_X_test)
accuracy_transformed = accuracy_score(y_test, y_pred_transformed)
print("Accuracy on transformed representations:", accuracy_transformed)

Accuracy on transformed representations: 0.084


In [223]:
def rotate_vector(vector, angle):
    """
    Rotate a 2D vector represented as [x, y] by a specified angle (in radians).
    
    Parameters:
        vector (list or numpy.ndarray): Input vector [x, y].
        angle (float): Angle of rotation in radians.
        
    Returns:
        numpy.ndarray: Rotated vector [x', y'].
    """
    # Convert angle to radians
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)
    
    # Create rotation matrix
    rotation_matrix = np.array([[cos_theta, -sin_theta],
                                [sin_theta, cos_theta]])
    
    # Convert vector to numpy array if it's not already
    vector = np.array(vector)
    
    # Apply rotation transformation
    rotated_vector = np.dot(rotation_matrix, vector)
    
    return rotated_vector

In [263]:
def apply_defense_transformation_2(X, padding_size=100):
    # shuffle transformation
    np.random.seed(51)
    np.random.shuffle(X)

    # rotate transformation X
    random_angle_degrees = random.randint(-15, 15)

    random_angle_radians = math.radians(random_angle_degrees)

    # for every two rows, rotate them by random amount
    for i in range(0, X.shape[0], 2):
        rotated = rotate_vector([X[i],X[i+1]], random_angle_radians)
        X[i] = rotated[0]
        X[i+1] = rotated[1]

    # add padding
    padding = np.ones((X.shape[0], padding_size))
    padding = np.random.normal(np.mean(X), np.std(X), X.shape[0]*padding_size).reshape(X.shape[0], padding_size)
    X = np.hstack((X, padding))
    
    np.random.shuffle(X)


    return X

In [375]:
def apply_defense_transformation_3(X,padding_size=100):

    data = np.load('../data/DefenseTransformationSubmit.npz')
    X = data['representations']
    n = X.shape[1]
    print(X.shape)

    matrix = np.random.rand(n,n)
    matrix = matrix/np.linalg.norm(matrix)*0.3+np.eye(n)

    X = np.dot(X, matrix)
    # add padding
    padding = np.ones((X.shape[0], padding_size))
    padding = np.random.normal(np.mean(X), np.std(X), X.shape[0]*padding_size).reshape(X.shape[0], padding_size)
    X = np.hstack((X, padding))
    return X

In [376]:
rep = apply_defense_transformation_3(data_sub["representations"])
print(rep.shape)

(20250, 192)
(20250, 292)


In [380]:
data_sub = np.load("../data/DefenseTransformationSubmit.npz")
print(data_sub["representations"].shape)
rep = apply_defense_transformation_2(data_sub["representations"])

# Convert from float64 to float32                                
rep = rep.astype(np.float32)
print(rep.shape)
                                  
# Apply defense transformation (e.g., your transformation)
np.savez(
    "../data/example_submission.npz",
    representations=rep
)

(20250, 192)
(20250, 292)


In [378]:
SERVER_URL = "http://34.71.138.79:9090"
TEAM_TOKEN = "l5pvMfL4ZID1QHmn"

In [379]:
def defense_submit(path_to_npz_file: str):
    endpoint = "/defense/submit"
    url = SERVER_URL + endpoint
    with open(path_to_npz_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})
        if response.status_code == 200:
            print("Request ok")
            print(response.json())
        else:
            raise Exception(
                f"Defense submit failed. Code: {response.status_code}, content: {response.json()}"
            )

In [386]:
defense_submit("../data/example_submission.npz")

Exception: Defense submit failed. Code: 400, content: {'detail': 'Exceeded submissions. Only 1/h allowed.'}

Function 1: {'score': 0.1600179374217987} (noise)
Function 2: {'score': 0.3573663830757141} (rotate + shuffle)
Function 3: {'score': 0.35843199491500854} (rotate + shuffle + padding)

Bag of ideas:
- polyfit
- mainfold